# 🚀 ADVANCED SQL

---

## 📋 **DAY 5 - LESSON 2: ADVANCED SQL**

### **🎯 MỤC TIÊU:**

1. **Window Functions** - ROW_NUMBER, RANK, LAG, LEAD
2. **Complex Joins** - Self-join, Cross join
3. **Set Operations** - UNION, INTERSECT, EXCEPT
4. **Pivot & Unpivot** - Transform data
5. **Advanced Aggregations** - ROLLUP, CUBE, GROUPING SETS
6. **Performance Tips** - Optimize SQL queries

---

## 💡 **ADVANCED SQL:**

- Window functions cho analytics
- Complex joins cho hierarchical data
- Set operations cho data comparison
- Pivot/Unpivot cho reporting
- Advanced aggregations cho subtotals

---

## 🔧 **SETUP**

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col, lit, when, desc, asc
from pyspark.sql.functions import row_number, rank, dense_rank, percent_rank, ntile
from pyspark.sql.functions import lag, lead, first, last
from pyspark.sql.functions import sum as spark_sum, avg as spark_avg, count as spark_count
from pyspark.sql.functions import round as spark_round
from pyspark.sql.window import Window
from pyspark.sql.types import *
import time
import random
from datetime import datetime, timedelta

spark = SparkSession.builder \
    .appName("AdvancedSQL") \
    .master("spark://spark-master:7077") \
    .config("spark.executor.memory", "2g") \
    .config("spark.driver.memory", "1g") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin123") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

print("✅ Spark Session Created")
print(f"Spark Version: {spark.version}")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/11 15:44:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


✅ Spark Session Created
Spark Version: 3.5.1


---

## 📊 **1. TẠO DATA MẪU**

In [2]:
print("="*80)
print("📊 1. GENERATING SAMPLE DATA")
print("="*80)

# Employees data
departments = ["Engineering", "Sales", "Marketing", "HR", "Finance"]
countries = ["USA", "UK", "Germany", "France", "Canada"]

print("\n🔹 Generating employees data...")

employees_data = []
for i in range(1, 501):
    dept = random.choice(departments)
    country = random.choice(countries)
    
    base_salary = {
        "Engineering": 80000,
        "Sales": 70000,
        "Marketing": 65000,
        "HR": 60000,
        "Finance": 75000
    }[dept]
    
    salary = base_salary + random.randint(-10000, 30000)
    
    # Manager ID (some employees have managers)
    manager_id = None
    if i > 50:  # First 50 are managers
        manager_id = f"EMP{random.randint(1, 50):04d}"
    
    employees_data.append((
        f"EMP{i:04d}",
        f"Employee {i}",
        random.randint(22, 60),
        dept,
        country,
        salary,
        manager_id,
        random.choice(["Active", "Active", "Active", "Inactive"]),
        (datetime(2020, 1, 1) + timedelta(days=random.randint(0, 1460))).strftime("%Y-%m-%d")
    ))

employees = spark.createDataFrame(employees_data,
    ["employee_id", "name", "age", "department", "country", "salary", "manager_id", "status", "hire_date"])

print(f"✅ Generated {employees.count():,} employees")
employees.show(5)

# Sales data
print("\n🔹 Generating sales data...")

sales_data = []
sale_id = 1
for month in range(1, 13):
    for day in [1, 15]:
        for emp_id in random.sample(range(1, 101), 20):
            sales_data.append((
                f"SALE{sale_id:05d}",
                f"EMP{emp_id:04d}",
                f"2024-{month:02d}-{day:02d}",
                random.randint(1000, 50000),
                random.choice(["Product A", "Product B", "Product C", "Product D"])
            ))
            sale_id += 1

sales = spark.createDataFrame(sales_data,
    ["sale_id", "employee_id", "sale_date", "amount", "product"])

print(f"✅ Generated {sales.count():,} sales records")
sales.show(5)

# Create temporary views
employees.createOrReplaceTempView("employees")
sales.createOrReplaceTempView("sales")

print("\n✅ Created temporary views: employees, sales")

📊 1. GENERATING SAMPLE DATA

🔹 Generating employees data...


✅ Generated 500 employees
+-----------+----------+---+----------+-------+------+----------+--------+----------+
|employee_id|      name|age|department|country|salary|manager_id|  status| hire_date|
+-----------+----------+---+----------+-------+------+----------+--------+----------+
|    EMP0001|Employee 1| 33|     Sales|    USA| 81958|      NULL|  Active|2022-07-30|
|    EMP0002|Employee 2| 43|        HR|Germany| 63411|      NULL|  Active|2020-12-12|
|    EMP0003|Employee 3| 22|   Finance| France| 70931|      NULL|Inactive|2021-01-11|
|    EMP0004|Employee 4| 33| Marketing|     UK| 60074|      NULL|  Active|2023-08-09|
|    EMP0005|Employee 5| 48| Marketing| Canada| 87114|      NULL|  Active|2022-06-20|
+-----------+----------+---+----------+-------+------+----------+--------+----------+
only showing top 5 rows


🔹 Generating sales data...
✅ Generated 480 sales records
+---------+-----------+----------+------+---------+
|  sale_id|employee_id| sale_date|amount|  product|
+---------+--

---

## 🪟 **2. WINDOW FUNCTIONS**

### **Window Functions là gì?**
- Tính toán trên một "cửa sổ" (window) của rows
- Không làm giảm số rows (khác với GROUP BY)
- Syntax: `function() OVER (PARTITION BY ... ORDER BY ...)`

### **Common Window Functions:**
- ROW_NUMBER(), RANK(), DENSE_RANK()
- LAG(), LEAD()
- SUM(), AVG(), COUNT() OVER (...)
- FIRST_VALUE(), LAST_VALUE()

In [3]:
print("="*80)
print("🪟 2. WINDOW FUNCTIONS")
print("="*80)

# A. Ranking Functions
print("\n📊 A. RANKING FUNCTIONS")
print("-" * 80)

result_rank = spark.sql("""
    SELECT 
        employee_id,
        name,
        department,
        salary,
        ROW_NUMBER() OVER (PARTITION BY department ORDER BY salary DESC) as row_num,
        RANK() OVER (PARTITION BY department ORDER BY salary DESC) as rank,
        DENSE_RANK() OVER (PARTITION BY department ORDER BY salary DESC) as dense_rank
    FROM employees
    WHERE status = 'Active'
    ORDER BY department, salary DESC
""")

print("Top 3 earners per department:")
result_rank.filter(col("row_num") <= 3).show(15)

print("""
💡 Difference:
   - ROW_NUMBER: 1, 2, 3, 4, 5 (unique)
   - RANK:       1, 2, 2, 4, 5 (skip after tie)
   - DENSE_RANK: 1, 2, 2, 3, 4 (no skip)
""")

🪟 2. WINDOW FUNCTIONS

📊 A. RANKING FUNCTIONS
--------------------------------------------------------------------------------
Top 3 earners per department:


+-----------+------------+-----------+------+-------+----+----------+
|employee_id|        name| department|salary|row_num|rank|dense_rank|
+-----------+------------+-----------+------+-------+----+----------+
|    EMP0322|Employee 322|Engineering|109070|      1|   1|         1|
|    EMP0338|Employee 338|Engineering|107167|      2|   2|         2|
|    EMP0323|Employee 323|Engineering|106531|      3|   3|         3|
|    EMP0446|Employee 446|    Finance|104748|      1|   1|         1|
|    EMP0203|Employee 203|    Finance|104252|      2|   2|         2|
|    EMP0339|Employee 339|    Finance|103886|      3|   3|         3|
|    EMP0493|Employee 493|         HR| 89950|      1|   1|         1|
|    EMP0213|Employee 213|         HR| 87834|      2|   2|         2|
|    EMP0411|Employee 411|         HR| 87120|      3|   3|         3|
|    EMP0397|Employee 397|  Marketing| 94545|      1|   1|         1|
|    EMP0200|Employee 200|  Marketing| 94533|      2|   2|         2|
|    EMP0126|Employe

In [4]:
# B. LAG and LEAD
print("\n📊 B. LAG and LEAD (Compare with previous/next row)")
print("-" * 80)

result_lag_lead = spark.sql("""
    SELECT 
        employee_id,
        name,
        department,
        salary,
        LAG(salary, 1) OVER (PARTITION BY department ORDER BY salary) as prev_salary,
        LEAD(salary, 1) OVER (PARTITION BY department ORDER BY salary) as next_salary,
        salary - LAG(salary, 1) OVER (PARTITION BY department ORDER BY salary) as diff_from_prev
    FROM employees
    WHERE status = 'Active'
    ORDER BY department, salary
""")

result_lag_lead.show(10)

print("""
💡 LAG and LEAD:
   - LAG(col, n):  Get value from n rows BEFORE
   - LEAD(col, n): Get value from n rows AFTER
   - Useful for: Time series, comparing with previous/next
""")


📊 B. LAG and LEAD (Compare with previous/next row)
--------------------------------------------------------------------------------
+-----------+------------+-----------+------+-----------+-----------+--------------+
|employee_id|        name| department|salary|prev_salary|next_salary|diff_from_prev|
+-----------+------------+-----------+------+-----------+-----------+--------------+
|    EMP0101|Employee 101|Engineering| 71279|       NULL|      71747|          NULL|
|    EMP0122|Employee 122|Engineering| 71747|      71279|      73136|           468|
|    EMP0369|Employee 369|Engineering| 73136|      71747|      73613|          1389|
|    EMP0489|Employee 489|Engineering| 73613|      73136|      75724|           477|
|    EMP0281|Employee 281|Engineering| 75724|      73613|      77262|          2111|
|    EMP0381|Employee 381|Engineering| 77262|      75724|      79116|          1538|
|    EMP0110|Employee 110|Engineering| 79116|      77262|      79542|          1854|
|    EMP0456|Empl

In [5]:
# C. Running Totals and Moving Averages
print("\n📊 C. RUNNING TOTALS and MOVING AVERAGES")
print("-" * 80)

result_running = spark.sql("""
    SELECT 
        sale_date,
        amount,
        SUM(amount) OVER (ORDER BY sale_date) as running_total,
        AVG(amount) OVER (ORDER BY sale_date ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) as moving_avg_3,
        COUNT(*) OVER (ORDER BY sale_date) as cumulative_count
    FROM sales
    ORDER BY sale_date
""")

result_running.show(20)

print("""
💡 Window Frame:
   - ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
     → Last 3 rows (including current)
   
   - ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
     → All rows from start to current (running total)
   
   - ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING
     → Previous, current, and next row
""")


📊 C. RUNNING TOTALS and MOVING AVERAGES
--------------------------------------------------------------------------------


26/01/11 15:44:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/11 15:44:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/11 15:44:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/11 15:44:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/11 15:44:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------+------+-------------+------------------+----------------+
| sale_date|amount|running_total|      moving_avg_3|cumulative_count|
+----------+------+-------------+------------------+----------------+
|2024-01-01| 22472|       538931|           22472.0|              20|
|2024-01-01| 31633|       538931|           27052.5|              20|
|2024-01-01| 34789|       538931|29631.333333333332|              20|
|2024-01-01| 29017|       538931|           31813.0|              20|
|2024-01-01| 28487|       538931|30764.333333333332|              20|
|2024-01-01| 23666|       538931|27056.666666666668|              20|
|2024-01-01| 22697|       538931|           24950.0|              20|
|2024-01-01| 42407|       538931|           29590.0|              20|
|2024-01-01| 14777|       538931|           26627.0|              20|
|2024-01-01| 13785|       538931|23656.333333333332|              20|
|2024-01-01| 36268|       538931|           21610.0|              20|
|2024-01-01| 26194| 

In [6]:
# D. FIRST_VALUE and LAST_VALUE
print("\n📊 D. FIRST_VALUE and LAST_VALUE")
print("-" * 80)

result_first_last = spark.sql("""
    SELECT 
        department,
        name,
        salary,
        FIRST_VALUE(name) OVER (PARTITION BY department ORDER BY salary DESC) as highest_paid,
        LAST_VALUE(name) OVER (PARTITION BY department ORDER BY salary DESC 
                               ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as lowest_paid,
        FIRST_VALUE(salary) OVER (PARTITION BY department ORDER BY salary DESC) as max_salary,
        salary - FIRST_VALUE(salary) OVER (PARTITION BY department ORDER BY salary DESC) as diff_from_max
    FROM employees
    WHERE status = 'Active'
    ORDER BY department, salary DESC
""")

result_first_last.show(15)

print("""
💡 FIRST_VALUE and LAST_VALUE:
   - FIRST_VALUE: First value in window
   - LAST_VALUE: Last value in window
   - Need ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
     for LAST_VALUE to work correctly
""")


📊 D. FIRST_VALUE and LAST_VALUE
--------------------------------------------------------------------------------
+-----------+------------+------+------------+------------+----------+-------------+
| department|        name|salary|highest_paid| lowest_paid|max_salary|diff_from_max|
+-----------+------------+------+------------+------------+----------+-------------+
|Engineering|Employee 322|109070|Employee 322|Employee 101|    109070|            0|
|Engineering|Employee 338|107167|Employee 322|Employee 101|    109070|        -1903|
|Engineering|Employee 323|106531|Employee 322|Employee 101|    109070|        -2539|
|Engineering|Employee 320|106428|Employee 322|Employee 101|    109070|        -2642|
|Engineering|Employee 180|106326|Employee 322|Employee 101|    109070|        -2744|
|Engineering| Employee 38|106167|Employee 322|Employee 101|    109070|        -2903|
|Engineering| Employee 64|104672|Employee 322|Employee 101|    109070|        -4398|
|Engineering|Employee 166|103831|Emp

In [7]:
# E. NTILE
print("\n📊 E. NTILE (Divide into buckets)")
print("-" * 80)

result_ntile = spark.sql("""
    SELECT 
        employee_id,
        name,
        department,
        salary,
        NTILE(4) OVER (PARTITION BY department ORDER BY salary DESC) as quartile,
        CASE 
            WHEN NTILE(4) OVER (PARTITION BY department ORDER BY salary DESC) = 1 THEN 'Top 25%'
            WHEN NTILE(4) OVER (PARTITION BY department ORDER BY salary DESC) = 2 THEN 'Top 50%'
            WHEN NTILE(4) OVER (PARTITION BY department ORDER BY salary DESC) = 3 THEN 'Top 75%'
            ELSE 'Bottom 25%'
        END as salary_bracket
    FROM employees
    WHERE status = 'Active'
    ORDER BY department, salary DESC
""")

result_ntile.show(20)

print("""
💡 NTILE(n):
   - Divide rows into n buckets
   - NTILE(4) = Quartiles (4 buckets)
   - NTILE(10) = Deciles (10 buckets)
   - NTILE(100) = Percentiles (100 buckets)
""")


📊 E. NTILE (Divide into buckets)
--------------------------------------------------------------------------------
+-----------+------------+-----------+------+--------+--------------+
|employee_id|        name| department|salary|quartile|salary_bracket|
+-----------+------------+-----------+------+--------+--------------+
|    EMP0322|Employee 322|Engineering|109070|       1|       Top 25%|
|    EMP0338|Employee 338|Engineering|107167|       1|       Top 25%|
|    EMP0323|Employee 323|Engineering|106531|       1|       Top 25%|
|    EMP0320|Employee 320|Engineering|106428|       1|       Top 25%|
|    EMP0180|Employee 180|Engineering|106326|       1|       Top 25%|
|    EMP0038| Employee 38|Engineering|106167|       1|       Top 25%|
|    EMP0064| Employee 64|Engineering|104672|       1|       Top 25%|
|    EMP0166|Employee 166|Engineering|103831|       1|       Top 25%|
|    EMP0270|Employee 270|Engineering|103587|       1|       Top 25%|
|    EMP0471|Employee 471|Engineering|103462|

In [8]:
# F. PERCENT_RANK
print("\n📊 F. PERCENT_RANK (Percentile)")
print("-" * 80)

result_percent = spark.sql("""
    SELECT 
        employee_id,
        name,
        department,
        salary,
        PERCENT_RANK() OVER (PARTITION BY department ORDER BY salary) as percentile,
        ROUND(PERCENT_RANK() OVER (PARTITION BY department ORDER BY salary) * 100, 2) as percentile_pct
    FROM employees
    WHERE status = 'Active'
    ORDER BY department, salary DESC
""")

result_percent.show(15)

print("""
💡 PERCENT_RANK():
   - Returns percentile (0.0 to 1.0)
   - 0.0 = lowest value
   - 1.0 = highest value
   - Multiply by 100 for percentage
""")


📊 F. PERCENT_RANK (Percentile)
--------------------------------------------------------------------------------
+-----------+------------+-----------+------+------------------+--------------+
|employee_id|        name| department|salary|        percentile|percentile_pct|
+-----------+------------+-----------+------+------------------+--------------+
|    EMP0322|Employee 322|Engineering|109070|               1.0|         100.0|
|    EMP0338|Employee 338|Engineering|107167|0.9852941176470589|         98.53|
|    EMP0323|Employee 323|Engineering|106531|0.9705882352941176|         97.06|
|    EMP0320|Employee 320|Engineering|106428|0.9558823529411765|         95.59|
|    EMP0180|Employee 180|Engineering|106326|0.9411764705882353|         94.12|
|    EMP0038| Employee 38|Engineering|106167|0.9264705882352942|         92.65|
|    EMP0064| Employee 64|Engineering|104672|0.9117647058823529|         91.18|
|    EMP0166|Employee 166|Engineering|103831|0.8970588235294118|         89.71|
|    EM

---

## 🔗 **3. COMPLEX JOINS**

In [9]:
print("="*80)
print("🔗 3. COMPLEX JOINS")
print("="*80)

# A. Self Join
print("\n📊 A. SELF JOIN (Employees and their managers)")
print("-" * 80)

result_self_join = spark.sql("""
    SELECT 
        e.employee_id,
        e.name as employee_name,
        e.department,
        e.salary as employee_salary,
        m.employee_id as manager_id,
        m.name as manager_name,
        m.salary as manager_salary
    FROM employees e
    LEFT JOIN employees m ON e.manager_id = m.employee_id
    WHERE e.status = 'Active'
    ORDER BY e.department, e.employee_id
""")

result_self_join.show(10)

print("""
💡 Self Join:
   - Join table with itself
   - Use aliases (e, m) to distinguish
   - Common use: Hierarchical data (employee-manager)
""")

🔗 3. COMPLEX JOINS

📊 A. SELF JOIN (Employees and their managers)
--------------------------------------------------------------------------------
+-----------+-------------+-----------+---------------+----------+------------+--------------+
|employee_id|employee_name| department|employee_salary|manager_id|manager_name|manager_salary|
+-----------+-------------+-----------+---------------+----------+------------+--------------+
|    EMP0027|  Employee 27|Engineering|          85504|      NULL|        NULL|          NULL|
|    EMP0033|  Employee 33|Engineering|          90435|      NULL|        NULL|          NULL|
|    EMP0037|  Employee 37|Engineering|          88755|      NULL|        NULL|          NULL|
|    EMP0038|  Employee 38|Engineering|         106167|      NULL|        NULL|          NULL|
|    EMP0040|  Employee 40|Engineering|          98947|      NULL|        NULL|          NULL|
|    EMP0048|  Employee 48|Engineering|          99595|      NULL|        NULL|          NULL

In [10]:
# B. Employees earning more than manager
print("\n📊 B. Employees earning more than their manager")
print("-" * 80)

result_higher_salary = spark.sql("""
    SELECT 
        e.employee_id,
        e.name as employee_name,
        e.salary as employee_salary,
        m.name as manager_name,
        m.salary as manager_salary,
        e.salary - m.salary as salary_diff
    FROM employees e
    INNER JOIN employees m ON e.manager_id = m.employee_id
    WHERE e.salary > m.salary
    ORDER BY salary_diff DESC
""")

result_higher_salary.show(10)


📊 B. Employees earning more than their manager
--------------------------------------------------------------------------------
+-----------+-------------+---------------+------------+--------------+-----------+
|employee_id|employee_name|employee_salary|manager_name|manager_salary|salary_diff|
+-----------+-------------+---------------+------------+--------------+-----------+
|    EMP0322| Employee 322|         109070| Employee 36|         59643|      49427|
|    EMP0240| Employee 240|         107751| Employee 22|         59347|      48404|
|    EMP0338| Employee 338|         107167|  Employee 4|         60074|      47093|
|    EMP0320| Employee 320|         106428|  Employee 4|         60074|      46354|
|    EMP0179| Employee 179|         102072| Employee 22|         59347|      42725|
|    EMP0247| Employee 247|          99978| Employee 22|         59347|      40631|
|    EMP0166| Employee 166|         103831| Employee 50|         63660|      40171|
|    EMP0346| Employee 346|    

In [11]:
# C. Cross Join
print("\n📊 C. CROSS JOIN (Cartesian product)")
print("-" * 80)

# Create small lookup table
categories = spark.createDataFrame([
    ("Category A",),
    ("Category B",),
    ("Category C",)
], ["category"])

categories.createOrReplaceTempView("categories")

result_cross = spark.sql("""
    SELECT 
        d.department,
        c.category
    FROM (SELECT DISTINCT department FROM employees) d
    CROSS JOIN categories c
    ORDER BY d.department, c.category
""")

print("All combinations of departments and categories:")
result_cross.show()

print("""
💡 Cross Join:
   - Cartesian product (all combinations)
   - Result size = table1_rows × table2_rows
   - ⚠️  Use carefully! Can be very large
   - Use case: Generate all combinations
""")


📊 C. CROSS JOIN (Cartesian product)
--------------------------------------------------------------------------------
All combinations of departments and categories:
+-----------+----------+
| department|  category|
+-----------+----------+
|Engineering|Category A|
|Engineering|Category B|
|Engineering|Category C|
|    Finance|Category A|
|    Finance|Category B|
|    Finance|Category C|
|         HR|Category A|
|         HR|Category B|
|         HR|Category C|
|  Marketing|Category A|
|  Marketing|Category B|
|  Marketing|Category C|
|      Sales|Category A|
|      Sales|Category B|
|      Sales|Category C|
+-----------+----------+


💡 Cross Join:
   - Cartesian product (all combinations)
   - Result size = table1_rows × table2_rows
   - ⚠️  Use carefully! Can be very large
   - Use case: Generate all combinations



In [12]:
# D. Multiple Joins
print("\n📊 D. MULTIPLE JOINS")
print("-" * 80)

result_multiple = spark.sql("""
    SELECT 
        e.employee_id,
        e.name,
        e.department,
        e.salary,
        COUNT(s.sale_id) as total_sales,
        COALESCE(SUM(s.amount), 0) as total_revenue,
        m.name as manager_name
    FROM employees e
    LEFT JOIN sales s ON e.employee_id = s.employee_id
    LEFT JOIN employees m ON e.manager_id = m.employee_id
    WHERE e.status = 'Active'
    GROUP BY e.employee_id, e.name, e.department, e.salary, m.name
    ORDER BY total_revenue DESC
""")

result_multiple.show(10)

print("""
💡 Multiple Joins:
   - Join multiple tables in one query
   - Use LEFT JOIN to keep all employees (even without sales)
   - Use COALESCE to handle NULLs
""")


📊 D. MULTIPLE JOINS
--------------------------------------------------------------------------------
+-----------+-----------+-----------+------+-----------+-------------+------------+
|employee_id|       name| department|salary|total_sales|total_revenue|manager_name|
+-----------+-----------+-----------+------+-----------+-------------+------------+
|    EMP0078|Employee 78|      Sales| 64876|          8|       318770|  Employee 8|
|    EMP0033|Employee 33|Engineering| 90435|          8|       223157|        NULL|
|    EMP0013|Employee 13|      Sales| 90946|          7|       214662|        NULL|
|    EMP0060|Employee 60|  Marketing| 66665|          8|       211087| Employee 38|
|    EMP0032|Employee 32|  Marketing| 88322|          7|       205087|        NULL|
|    EMP0057|Employee 57|    Finance| 92738|          7|       202772| Employee 43|
|    EMP0056|Employee 56|  Marketing| 89386|          6|       202474|  Employee 9|
|    EMP0018|Employee 18|      Sales| 88504|          7|  

---

## 🔀 **4. SET OPERATIONS**

### **Set Operations:**
- **UNION**: Combine results (remove duplicates)
- **UNION ALL**: Combine results (keep duplicates)
- **INTERSECT**: Common rows
- **EXCEPT**: Rows in first but not in second

In [13]:
print("="*80)
print("🔀 4. SET OPERATIONS")
print("="*80)

# A. UNION
print("\n📊 A. UNION (Combine results)")
print("-" * 80)

result_union = spark.sql("""
    SELECT employee_id, name, 'High Earner' as category
    FROM employees
    WHERE salary > 85000
    
    UNION
    
    SELECT employee_id, name, 'Young Employee' as category
    FROM employees
    WHERE age < 30
    
    ORDER BY employee_id
""")

result_union.show(10)

🔀 4. SET OPERATIONS

📊 A. UNION (Combine results)
--------------------------------------------------------------------------------
+-----------+-----------+--------------+
|employee_id|       name|      category|
+-----------+-----------+--------------+
|    EMP0003| Employee 3|Young Employee|
|    EMP0005| Employee 5|   High Earner|
|    EMP0007| Employee 7|   High Earner|
|    EMP0008| Employee 8|   High Earner|
|    EMP0009| Employee 9|   High Earner|
|    EMP0012|Employee 12|   High Earner|
|    EMP0012|Employee 12|Young Employee|
|    EMP0013|Employee 13|   High Earner|
|    EMP0014|Employee 14|   High Earner|
|    EMP0015|Employee 15|   High Earner|
+-----------+-----------+--------------+
only showing top 10 rows



In [14]:
# B. UNION ALL
print("\n📊 B. UNION ALL (Keep duplicates)")
print("-" * 80)

result_union_all = spark.sql("""
    SELECT employee_id, name, 'High Earner' as category
    FROM employees
    WHERE salary > 85000
    
    UNION ALL
    
    SELECT employee_id, name, 'Young Employee' as category
    FROM employees
    WHERE age < 30
    
    ORDER BY employee_id
""")

print(f"UNION result: {result_union.count()} rows (duplicates removed)")
print(f"UNION ALL result: {result_union_all.count()} rows (duplicates kept)")
result_union_all.show(10)


📊 B. UNION ALL (Keep duplicates)
--------------------------------------------------------------------------------
UNION result: 318 rows (duplicates removed)
UNION ALL result: 318 rows (duplicates kept)
+-----------+-----------+--------------+
|employee_id|       name|      category|
+-----------+-----------+--------------+
|    EMP0003| Employee 3|Young Employee|
|    EMP0005| Employee 5|   High Earner|
|    EMP0007| Employee 7|   High Earner|
|    EMP0008| Employee 8|   High Earner|
|    EMP0009| Employee 9|   High Earner|
|    EMP0012|Employee 12|Young Employee|
|    EMP0012|Employee 12|   High Earner|
|    EMP0013|Employee 13|   High Earner|
|    EMP0014|Employee 14|   High Earner|
|    EMP0015|Employee 15|   High Earner|
+-----------+-----------+--------------+
only showing top 10 rows



In [15]:
# C. INTERSECT
print("\n📊 C. INTERSECT (Common rows)")
print("-" * 80)

result_intersect = spark.sql("""
    SELECT employee_id, name
    FROM employees
    WHERE salary > 80000
    
    INTERSECT
    
    SELECT employee_id, name
    FROM employees
    WHERE age < 35
""")

print("Employees who are both high earners AND young:")
result_intersect.show(10)


📊 C. INTERSECT (Common rows)
--------------------------------------------------------------------------------
Employees who are both high earners AND young:
+-----------+------------+
|employee_id|        name|
+-----------+------------+
|    EMP0392|Employee 392|
|    EMP0455|Employee 455|
|    EMP0409|Employee 409|
|    EMP0423|Employee 423|
|    EMP0319|Employee 319|
|    EMP0149|Employee 149|
|    EMP0170|Employee 170|
|    EMP0362|Employee 362|
|    EMP0118|Employee 118|
|    EMP0396|Employee 396|
+-----------+------------+
only showing top 10 rows



In [16]:
# D. EXCEPT
print("\n📊 D. EXCEPT (Difference)")
print("-" * 80)

result_except = spark.sql("""
    SELECT employee_id, name, salary
    FROM employees
    WHERE department = 'Engineering'
    
    EXCEPT
    
    SELECT employee_id, name, salary
    FROM employees
    WHERE salary > 85000
""")

print("Engineering employees who are NOT high earners:")
result_except.show(10)

print("""
💡 Set Operations Summary:
   - UNION: A ∪ B (remove duplicates)
   - UNION ALL: A + B (keep duplicates)
   - INTERSECT: A ∩ B (common)
   - EXCEPT: A - B (difference)
   
   ⚠️  All queries must have same number of columns and types
""")


📊 D. EXCEPT (Difference)
--------------------------------------------------------------------------------
Engineering employees who are NOT high earners:
+-----------+------------+------+
|employee_id|        name|salary|
+-----------+------------+------+
|    EMP0052| Employee 52| 81918|
|    EMP0215|Employee 215| 82603|
|    EMP0197|Employee 197| 81704|
|    EMP0122|Employee 122| 71747|
|    EMP0227|Employee 227| 81633|
|    EMP0235|Employee 235| 76550|
|    EMP0080| Employee 80| 75704|
|    EMP0121|Employee 121| 70183|
|    EMP0110|Employee 110| 79116|
|    EMP0142|Employee 142| 84455|
+-----------+------------+------+
only showing top 10 rows


💡 Set Operations Summary:
   - UNION: A ∪ B (remove duplicates)
   - UNION ALL: A + B (keep duplicates)
   - INTERSECT: A ∩ B (common)
   - EXCEPT: A - B (difference)
   
   ⚠️  All queries must have same number of columns and types



---

## 🔄 **5. PIVOT & UNPIVOT**

In [17]:
print("="*80)
print("🔄 5. PIVOT & UNPIVOT")
print("="*80)

# A. PIVOT
print("\n📊 A. PIVOT (Transform rows to columns)")
print("-" * 80)

print("Original data (long format):")
spark.sql("""
    SELECT department, country, COUNT(*) as employee_count
    FROM employees
    GROUP BY department, country
    ORDER BY department, country
""").show(10)

print("\nPivoted data (wide format):")
result_pivot = spark.sql("""
    SELECT * FROM (
        SELECT department, country, salary
        FROM employees
    )
    PIVOT (
        COUNT(*) as count,
        ROUND(AVG(salary), 2) as avg_salary
        FOR country IN ('USA', 'UK', 'Germany', 'France', 'Canada')
    )
    ORDER BY department
""")

result_pivot.show()

print("""
💡 PIVOT:
   - Transform rows to columns
   - Syntax: PIVOT (agg_func FOR column IN (values))
   - Useful for: Reports, dashboards
   - Long format → Wide format
""")

🔄 5. PIVOT & UNPIVOT

📊 A. PIVOT (Transform rows to columns)
--------------------------------------------------------------------------------
Original data (long format):
+-----------+-------+--------------+
| department|country|employee_count|
+-----------+-------+--------------+
|Engineering| Canada|            19|
|Engineering| France|            22|
|Engineering|Germany|            13|
|Engineering|     UK|            19|
|Engineering|    USA|            26|
|    Finance| Canada|            26|
|    Finance| France|            30|
|    Finance|Germany|            15|
|    Finance|     UK|            17|
|    Finance|    USA|            15|
+-----------+-------+--------------+
only showing top 10 rows


Pivoted data (wide format):
+-----------+---------+--------------+--------+-------------+-------------+------------------+------------+-----------------+------------+-----------------+
| department|USA_count|USA_avg_salary|UK_count|UK_avg_salary|Germany_count|Germany_avg_salary|Franc

In [18]:
# B. PIVOT using CASE WHEN
print("\n📊 B. PIVOT using CASE WHEN (more flexible)")
print("-" * 80)

result_pivot_case = spark.sql("""
    SELECT 
        department,
        COUNT(*) as total_employees,
        SUM(CASE WHEN country = 'USA' THEN 1 ELSE 0 END) as usa_count,
        SUM(CASE WHEN country = 'UK' THEN 1 ELSE 0 END) as uk_count,
        SUM(CASE WHEN country = 'Germany' THEN 1 ELSE 0 END) as germany_count,
        SUM(CASE WHEN country = 'France' THEN 1 ELSE 0 END) as france_count,
        SUM(CASE WHEN country = 'Canada' THEN 1 ELSE 0 END) as canada_count,
        ROUND(AVG(CASE WHEN country = 'USA' THEN salary END), 2) as usa_avg_salary,
        ROUND(AVG(CASE WHEN country = 'UK' THEN salary END), 2) as uk_avg_salary
    FROM employees
    GROUP BY department
    ORDER BY department
""")

result_pivot_case.show()

print("""
💡 PIVOT with CASE WHEN:
   - More flexible than PIVOT
   - Can use any condition
   - Can mix different aggregations
   - Easier to understand
""")


📊 B. PIVOT using CASE WHEN (more flexible)
--------------------------------------------------------------------------------
+-----------+---------------+---------+--------+-------------+------------+------------+--------------+-------------+
| department|total_employees|usa_count|uk_count|germany_count|france_count|canada_count|usa_avg_salary|uk_avg_salary|
+-----------+---------------+---------+--------+-------------+------------+------------+--------------+-------------+
|Engineering|             99|       26|      19|           13|          22|          19|      87948.96|     90710.79|
|    Finance|            103|       15|      17|           15|          30|          26|       81691.0|     84926.47|
|         HR|             89|       13|      18|           21|          22|          15|      68408.08|     74094.78|
|  Marketing|            101|       32|      15|           19|          14|          21|      73564.09|     73648.73|
|      Sales|            108|       20|      27| 

In [19]:
# C. UNPIVOT
print("\n📊 C. UNPIVOT (Transform columns to rows)")
print("-" * 80)

# First create a pivoted table
pivoted = spark.sql("""
    SELECT 
        department,
        SUM(CASE WHEN country = 'USA' THEN 1 ELSE 0 END) as USA,
        SUM(CASE WHEN country = 'UK' THEN 1 ELSE 0 END) as UK,
        SUM(CASE WHEN country = 'Germany' THEN 1 ELSE 0 END) as Germany
    FROM employees
    GROUP BY department
""")

pivoted.createOrReplaceTempView("pivoted_data")

print("Pivoted data (wide format):")
pivoted.show()

print("\nUnpivoted data (long format):")
result_unpivot = spark.sql("""
    SELECT department, country, employee_count
    FROM pivoted_data
    LATERAL VIEW STACK(3,
        'USA', USA,
        'UK', UK,
        'Germany', Germany
    ) AS country, employee_count
    ORDER BY department, country
""")

result_unpivot.show()

print("""
💡 UNPIVOT:
   - Transform columns to rows
   - Use LATERAL VIEW STACK
   - Wide format → Long format
   - Useful for: Normalizing data
""")


📊 C. UNPIVOT (Transform columns to rows)
--------------------------------------------------------------------------------
Pivoted data (wide format):
+-----------+---+---+-------+
| department|USA| UK|Germany|
+-----------+---+---+-------+
|      Sales| 20| 27|     24|
|Engineering| 26| 19|     13|
|         HR| 13| 18|     21|
|    Finance| 15| 17|     15|
|  Marketing| 32| 15|     19|
+-----------+---+---+-------+


Unpivoted data (long format):
+-----------+-------+--------------+
| department|country|employee_count|
+-----------+-------+--------------+
|Engineering|Germany|            13|
|Engineering|     UK|            19|
|Engineering|    USA|            26|
|    Finance|Germany|            15|
|    Finance|     UK|            17|
|    Finance|    USA|            15|
|         HR|Germany|            21|
|         HR|     UK|            18|
|         HR|    USA|            13|
|  Marketing|Germany|            19|
|  Marketing|     UK|            15|
|  Marketing|    USA|        

---

## 📈 **6. ADVANCED AGGREGATIONS**

In [20]:
print("="*80)
print("📈 6. ADVANCED AGGREGATIONS")
print("="*80)

# A. GROUPING SETS
print("\n📊 A. GROUPING SETS (Multiple GROUP BY in one query)")
print("-" * 80)

result_grouping_sets = spark.sql("""
    SELECT 
        department,
        country,
        COUNT(*) as employee_count,
        ROUND(AVG(salary), 2) as avg_salary
    FROM employees
    WHERE status = 'Active'
    GROUP BY GROUPING SETS (
        (department, country),
        (department),
        (country),
        ()
    )
    ORDER BY department NULLS LAST, country NULLS LAST
""")

result_grouping_sets.show(30)

print("""
💡 GROUPING SETS:
   - Multiple GROUP BY in one query
   - More efficient than UNION
   - NULL means "all" (subtotal/grand total)
""")

📈 6. ADVANCED AGGREGATIONS

📊 A. GROUPING SETS (Multiple GROUP BY in one query)
--------------------------------------------------------------------------------
+-----------+-------+--------------+----------+
| department|country|employee_count|avg_salary|
+-----------+-------+--------------+----------+
|Engineering| Canada|            16|  95807.94|
|Engineering| France|            15|  92900.27|
|Engineering|Germany|             9|  91312.56|
|Engineering|     UK|            13|  95112.15|
|Engineering|    USA|            16|  88251.63|
|Engineering|   NULL|            69|   92706.2|
|    Finance| Canada|            23|   84621.0|
|    Finance| France|            25|   83208.4|
|    Finance|Germany|            11|  78902.27|
|    Finance|     UK|            11|  88459.09|
|    Finance|    USA|            14|  81014.43|
|    Finance|   NULL|            84|  83353.21|
|         HR| Canada|            10|   68230.9|
|         HR| France|            13|  63171.69|
|         HR|Germany|  

In [21]:
# B. ROLLUP
print("\n📊 B. ROLLUP (Hierarchical subtotals)")
print("-" * 80)

result_rollup = spark.sql("""
    SELECT 
        department,
        country,
        COUNT(*) as employee_count,
        ROUND(AVG(salary), 2) as avg_salary,
        ROUND(SUM(salary), 2) as total_salary
    FROM employees
    WHERE status = 'Active'
    GROUP BY ROLLUP(department, country)
    ORDER BY department NULLS LAST, country NULLS LAST
""")

result_rollup.show(30)

print("""
💡 ROLLUP:
   - Hierarchical aggregation
   - ROLLUP(A, B) creates:
     * (A, B) - Detail level
     * (A)    - Subtotal by A
     * ()     - Grand total
   - Useful for: Reports with subtotals
""")


📊 B. ROLLUP (Hierarchical subtotals)
--------------------------------------------------------------------------------
+-----------+-------+--------------+----------+------------+
| department|country|employee_count|avg_salary|total_salary|
+-----------+-------+--------------+----------+------------+
|Engineering| Canada|            16|  95807.94|     1532927|
|Engineering| France|            15|  92900.27|     1393504|
|Engineering|Germany|             9|  91312.56|      821813|
|Engineering|     UK|            13|  95112.15|     1236458|
|Engineering|    USA|            16|  88251.63|     1412026|
|Engineering|   NULL|            69|   92706.2|     6396728|
|    Finance| Canada|            23|   84621.0|     1946283|
|    Finance| France|            25|   83208.4|     2080210|
|    Finance|Germany|            11|  78902.27|      867925|
|    Finance|     UK|            11|  88459.09|      973050|
|    Finance|    USA|            14|  81014.43|     1134202|
|    Finance|   NULL|      

In [22]:
# C. CUBE
print("\n📊 C. CUBE (All combinations)")
print("-" * 80)

result_cube = spark.sql("""
    SELECT 
        department,
        country,
        COUNT(*) as employee_count,
        ROUND(AVG(salary), 2) as avg_salary
    FROM employees
    WHERE status = 'Active'
    GROUP BY CUBE(department, country)
    ORDER BY department NULLS LAST, country NULLS LAST
""")

result_cube.show(40)

print("""
💡 CUBE:
   - All possible combinations
   - CUBE(A, B) creates:
     * (A, B) - Both
     * (A)    - A only
     * (B)    - B only
     * ()     - Grand total
   - More combinations than ROLLUP
""")


📊 C. CUBE (All combinations)
--------------------------------------------------------------------------------
+-----------+-------+--------------+----------+
| department|country|employee_count|avg_salary|
+-----------+-------+--------------+----------+
|Engineering| Canada|            16|  95807.94|
|Engineering| France|            15|  92900.27|
|Engineering|Germany|             9|  91312.56|
|Engineering|     UK|            13|  95112.15|
|Engineering|    USA|            16|  88251.63|
|Engineering|   NULL|            69|   92706.2|
|    Finance| Canada|            23|   84621.0|
|    Finance| France|            25|   83208.4|
|    Finance|Germany|            11|  78902.27|
|    Finance|     UK|            11|  88459.09|
|    Finance|    USA|            14|  81014.43|
|    Finance|   NULL|            84|  83353.21|
|         HR| Canada|            10|   68230.9|
|         HR| France|            13|  63171.69|
|         HR|Germany|            18|  73553.28|
|         HR|     UK|    

In [23]:
# D. GROUPING function
print("\n📊 D. GROUPING function (Identify subtotal rows)")
print("-" * 80)

result_grouping = spark.sql("""
    SELECT 
        department,
        country,
        COUNT(*) as employee_count,
        ROUND(AVG(salary), 2) as avg_salary,
        GROUPING(department) as is_dept_total,
        GROUPING(country) as is_country_total,
        CASE 
            WHEN GROUPING(department) = 1 AND GROUPING(country) = 1 THEN 'Grand Total'
            WHEN GROUPING(department) = 1 THEN 'Country Total'
            WHEN GROUPING(country) = 1 THEN 'Department Total'
            ELSE 'Detail'
        END as level
    FROM employees
    WHERE status = 'Active'
    GROUP BY ROLLUP(department, country)
    ORDER BY department NULLS LAST, country NULLS LAST
""")

result_grouping.show(30)

print("""
💡 GROUPING function:
   - Returns 1 if column is aggregated (NULL is subtotal)
   - Returns 0 if column is not aggregated (NULL is real NULL)
   - Useful for: Identifying subtotal rows
""")


📊 D. GROUPING function (Identify subtotal rows)
--------------------------------------------------------------------------------
+-----------+-------+--------------+----------+-------------+----------------+----------------+
| department|country|employee_count|avg_salary|is_dept_total|is_country_total|           level|
+-----------+-------+--------------+----------+-------------+----------------+----------------+
|Engineering| Canada|            16|  95807.94|            0|               0|          Detail|
|Engineering| France|            15|  92900.27|            0|               0|          Detail|
|Engineering|Germany|             9|  91312.56|            0|               0|          Detail|
|Engineering|     UK|            13|  95112.15|            0|               0|          Detail|
|Engineering|    USA|            16|  88251.63|            0|               0|          Detail|
|Engineering|   NULL|            69|   92706.2|            0|               1|Department Total|
|    F

---

## ⚡ **7. PERFORMANCE TIPS**

In [24]:
print("="*80)
print("⚡ 7. SQL PERFORMANCE TIPS")
print("="*80)

print("""
💡 PERFORMANCE TIPS:

1. FILTER EARLY (Predicate Pushdown)
   ✅ Good:
      SELECT * FROM large_table
      WHERE date = '2024-01-01'
      
   ❌ Bad:
      SELECT * FROM (
          SELECT * FROM large_table
      ) WHERE date = '2024-01-01'

2. SELECT ONLY NEEDED COLUMNS (Column Pruning)
   ✅ Good:
      SELECT id, name, salary FROM employees
      
   ❌ Bad:
      SELECT * FROM employees

3. USE BROADCAST JOIN for small tables
   ✅ Good:
      SELECT /*+ BROADCAST(small_table) */
      FROM large_table JOIN small_table
      
4. AVOID CROSS JOIN
   ❌ Bad:
      SELECT * FROM table1 CROSS JOIN table2
      
   ✅ Good:
      SELECT * FROM table1 JOIN table2 ON table1.id = table2.id

5. USE WINDOW FUNCTIONS instead of self-joins
   ✅ Good:
      SELECT *, ROW_NUMBER() OVER (PARTITION BY dept ORDER BY salary DESC)
      
   ❌ Bad:
      SELECT * FROM employees e1
      WHERE (SELECT COUNT(*) FROM employees e2 
             WHERE e2.dept = e1.dept AND e2.salary > e1.salary) < 3

6. CACHE frequently used tables
   ✅ Good:
      CACHE TABLE employees;
      UNCACHE TABLE employees;

7. USE EXPLAIN to check execution plan
   ✅ Always:
      EXPLAIN SELECT * FROM ...
""")

# Demo: Broadcast hint
print("\n📊 DEMO: Broadcast Join Hint")
print("-" * 80)

# Small lookup table
lookup = spark.createDataFrame([
    ("Engineering", "Tech"),
    ("Sales", "Business"),
    ("Marketing", "Business"),
    ("HR", "Support"),
    ("Finance", "Support")
], ["department", "division"])

lookup.createOrReplaceTempView("lookup")

# Without broadcast hint
print("Without broadcast hint:")
result_no_hint = spark.sql("""
    SELECT e.name, e.department, l.division
    FROM employees e
    JOIN lookup l ON e.department = l.department
""")
result_no_hint.explain()

# With broadcast hint
print("\nWith broadcast hint:")
result_with_hint = spark.sql("""
    SELECT /*+ BROADCAST(l) */ e.name, e.department, l.division
    FROM employees e
    JOIN lookup l ON e.department = l.department
""")
result_with_hint.explain()

print("""
💡 Broadcast Hint:
   - Use /*+ BROADCAST(table) */ before SELECT
   - Forces small table to be broadcast
   - Avoids shuffle
   - Much faster for small dimension tables
""")

⚡ 7. SQL PERFORMANCE TIPS

💡 PERFORMANCE TIPS:

1. FILTER EARLY (Predicate Pushdown)
   ✅ Good:
      SELECT * FROM large_table
      WHERE date = '2024-01-01'
      
   ❌ Bad:
      SELECT * FROM (
          SELECT * FROM large_table
      ) WHERE date = '2024-01-01'

2. SELECT ONLY NEEDED COLUMNS (Column Pruning)
   ✅ Good:
      SELECT id, name, salary FROM employees
      
   ❌ Bad:
      SELECT * FROM employees

3. USE BROADCAST JOIN for small tables
   ✅ Good:
      SELECT /*+ BROADCAST(small_table) */
      FROM large_table JOIN small_table
      
4. AVOID CROSS JOIN
   ❌ Bad:
      SELECT * FROM table1 CROSS JOIN table2
      
   ✅ Good:
      SELECT * FROM table1 JOIN table2 ON table1.id = table2.id

5. USE WINDOW FUNCTIONS instead of self-joins
   ✅ Good:
      SELECT *, ROW_NUMBER() OVER (PARTITION BY dept ORDER BY salary DESC)
      
   ❌ Bad:
      SELECT * FROM employees e1
      WHERE (SELECT COUNT(*) FROM employees e2 
             WHERE e2.dept = e1.dept AND e2.salary 

---

## 🎓 **KEY TAKEAWAYS**

### **✅ What You Learned:**

1. **Window Functions**
   - ROW_NUMBER(), RANK(), DENSE_RANK()
   - LAG(), LEAD()
   - Running totals, moving averages
   - FIRST_VALUE(), LAST_VALUE()
   - NTILE(), PERCENT_RANK()

2. **Complex Joins**
   - Self Join (hierarchical data)
   - Cross Join (cartesian product)
   - Multiple Joins

3. **Set Operations**
   - UNION / UNION ALL
   - INTERSECT
   - EXCEPT

4. **Pivot & Unpivot**
   - PIVOT: Rows → Columns
   - UNPIVOT: Columns → Rows (LATERAL VIEW STACK)
   - CASE WHEN for flexible pivoting

5. **Advanced Aggregations**
   - GROUPING SETS
   - ROLLUP (hierarchical)
   - CUBE (all combinations)
   - GROUPING() function

6. **Performance Tips**
   - Filter early
   - Select only needed columns
   - Use broadcast join for small tables
   - Avoid cross join
   - Use window functions instead of self-joins
   - Cache frequently used tables

### **📊 Quick Reference:**

```sql
-- Window Function
SELECT 
    col1,
    ROW_NUMBER() OVER (PARTITION BY col2 ORDER BY col3) as rn
FROM table

-- Self Join
SELECT e.*, m.name as manager_name
FROM employees e
LEFT JOIN employees m ON e.manager_id = m.employee_id

-- Pivot
SELECT * FROM table
PIVOT (
    COUNT(*) FOR country IN ('USA', 'UK')
)

-- Rollup
SELECT dept, country, COUNT(*)
FROM employees
GROUP BY ROLLUP(dept, country)
```

### **🚀 Next:** Day 5 - Lesson 3: Catalog & Metadata

---

In [25]:
# Cleanup
spark.catalog.clearCache()
spark.stop()

print("✅ Spark session stopped")
print("\n🎉 DAY 5 - LESSON 2 COMPLETED!")
print("\n💡 Remember:")
print("   - Window functions are powerful for analytics")
print("   - Use ROLLUP/CUBE for reports with subtotals")
print("   - Broadcast small tables for better performance")
print("   - EXPLAIN your queries to check execution plan")
print("   - SQL and DataFrame API have same performance!")
print("\n🔥 Quote: 'Window functions: The Swiss Army knife of SQL!' 🪟")

✅ Spark session stopped

🎉 DAY 5 - LESSON 2 COMPLETED!

💡 Remember:
   - Window functions are powerful for analytics
   - Use ROLLUP/CUBE for reports with subtotals
   - Broadcast small tables for better performance
   - EXPLAIN your queries to check execution plan
   - SQL and DataFrame API have same performance!

🔥 Quote: 'Window functions: The Swiss Army knife of SQL!' 🪟
